In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE180394'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Tubular Transcriptome in the Cprobe Cohort"
!Series_summary	"We used microarrays to analyze the transcriptome of microdissected renal biopsies from patients with kidney disease and living donors. We derived pathway specific scores for Angiopoietin-Tie signaling pathway activation at mRNA level (or transcriptome level) for individual patients and studied the association of pathway activation with disease outcomes."
!Series_overall_design	"Tubular gene expression data from micro dissected human kidney biopsy samples  from patients with chronic kidney disease(Lupus, DN, IgA,HT, TN) and healthy living donors."
!Series_overall_design	"Profiling was performed on Affymetrix ST2.1 microarray platform. "
Sample Characteristics Dictionary:
{0: ['sample group: Living donor', "sample group: 2' FSGS", 'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL', 'sample group: DN', 'sample group: FGGS', 'sample group: FSGS', 'sample group: Hydronep

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True

# Determine the rows for each trait
sample_characteristics_dict = {
    0: ['sample group: Living donor', "sample group: 2' FSGS", 'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL', 
        'sample group: DN', 'sample group: FGGS', 'sample group: FSGS', 'sample group: Hydronephrosis', 'sample group: IgAN', 
        'sample group: Interstitial nephritis', 'sample group: Hypertensive Nephrosclerosis', 'sample group: Light-Chain Deposit Disease (IgG lambda)', 
        'sample group: LN-WHO III', 'sample group: LN-WHO III+V', 'sample group: LN-WHO IV', 'sample group: LN-WHO IV+V', 'sample group: LN-WHO V', 
        'sample group: LN-WHO-I/II', 'sample group: MCD', 'sample group: MN', 'sample group: CKD with mod-severe Interstitial fibrosis', 
        'sample group: Thin-BMD', 'sample group: Unaffected parts of Tumor Nephrectomy'], 
    1: ['tissue: Tubuli from kidney biopsy']
}

# Assuming dataset contains information in a structured manner and we could infer keys (abstract example)
trait_row = 0        # Set to 0 if CKD status is derived from sample group info.
age_row = None       # No age information apparent in the dataset
gender_row = None    # No gender information apparent in the dataset

# Define data conversion functions
def convert_trait(value):
    if 'donor' in value.lower() or 'unaffected' in value.lower():
        return 0  # Non-CKD
    elif 'ckd' in value.lower() or 'chronic' in value.lower() or 'gn' in value.lower() or 'disease' in value.lower():
        return 1  # CKD
    return None

def convert_age(value):
    # Age information not available
    return None

def convert_gender(value):
    # Gender information not available
    return None

# Save cohort information
save_cohort_info('GSE180394', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, process it
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE180394.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5607814': [0], 'GSM5607815': [0], 'GSM5607816': [0], 'GSM5607817': [0], 'GSM5607818': [0], 'GSM5607819': [0], 'GSM5607820': [0], 'GSM5607821': [0], 'GSM5607822': [0], 'GSM5607823': [None], 'GSM5607824': [1], 'GSM5607825': [None], 'GSM5607826': [None], 'GSM5607827': [None], 'GSM5607828': [None], 'GSM5607829': [None], 'GSM5607830': [None], 'GSM5607831': [None], 'GSM5607832': [None], 'GSM5607833': [None], 'GSM5607834': [None], 'GSM5607835': [None], 'GSM5607836': [None], 'GSM5607837': [None], 'GSM5607838': [None], 'GSM5607839': [None], 'GSM5607840': [None], 'GSM5607841': [None], 'GSM5607842': [None], 'GSM5607843': [None], 'GSM5607844': [None], 'GSM5607845': [1], 'GSM5607846': [None], 'GSM5607847': [None], 'GSM5607848': [None], 'GSM5607849': [None], 'GSM5607850': [None], 'GSM5607851': [None], 'GSM5607852': [None], 'GSM5607853': [None], 'GSM5607854': [None], 'GSM5607855': [None], 'GSM5607856': [None], 'GSM5607857': [None], 'GSM5607858': [None], 'GSM5607859': [None], 'GSM5607860': [None]

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identifying the keys for mapping
identifier_key = 'ID'
gene_symbol_key = 'ENTREZ_GENE_ID'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)
